In [2]:
import pandas as pd
import numpy as np


In [3]:
#read large_eve.json and create a dataframe
df = pd.read_json('large_eve.json', lines=True)
df

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


In [4]:
#show all columns of df
df.columns

Index(['timestamp', 'flow_id', 'pcap_cnt', 'event_type', 'vlan', 'src_ip',
       'src_port', 'dest_ip', 'dest_port', 'proto', 'alert', 'ssh', 'dns',
       'tx_id', 'http', 'app_proto', 'fileinfo', 'tls', 'vars', 'flow',
       'icmp_type', 'icmp_code', 'tcp', 'smtp', 'email', 'app_proto_tc',
       'app_proto_ts', 'stats'],
      dtype='object')

In [9]:
df['proto'].unique()

array(['TCP', 'UDP', 'IPv6-ICMP', nan], dtype=object)

In [8]:
df[df['dns'].notna()]

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716752,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.172,60654.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716753,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.172,60654.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716893,2017-07-22T19:47:38.146503-0500,2.199316e+15,4452917.0,dns,130.0,192.168.203.67,50975.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716894,2017-07-22T19:47:38.146953-0500,2.199316e+15,4452918.0,dns,130.0,192.168.207.4,53.0,192.168.203.67,50975.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
